In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline


In [2]:
import matplotlib.pyplot as plt
plt.style.use('dark_background')

import numpy as np
import yaml
import os
os.chdir("../")
from datasets.CrossDomain import CrossDomain
from datasets.Coco import Coco


In [20]:
with open("/home/topkech/work/pytorch-superpoint/configs/cross_domain_train.yaml", "r") as f:
# with open("/home/topkech/work/pytorch-superpoint/configs/superpoint_coco_train_heatmap.yaml", "r") as f:
    config = yaml.safe_load(f)

cd = CrossDomain(task="train", **config["data"])
print(cd[0].keys())
fig, axs = plt.subplots(2, 5, figsize=(20,10))
for axi, i in enumerate(range(-5, 0)):
    ex = cd[i]
    im = ex["image"][0][...,np.newaxis]
    im = np.dstack([1-ex["valid_mask"].squeeze()[..., np.newaxis], ex["labels_2D_gaussian"][0][...,np.newaxis], im])

    axs[0][axi].imshow(im)
    warped_im = ex["warped_img"][0][...,np.newaxis]
    warped_im = np.dstack([1-ex["warped_valid_mask"].squeeze()[..., np.newaxis], ex["warped_labels_gaussian"][0][...,np.newaxis], warped_im])
    axs[1][axi].imshow(warped_im)
    axs[1][axi].set_title(ex["name"])
    axs[0][axi].set_axis_off()
    axs[1][axi].set_axis_off()
plt.tight_layout()


(1, 5)
load labels from:  logs/trash/predictions/train


FileNotFoundError: [Errno 2] No such file or directory: 'logs/trash/predictions/train/20221114_114742_ssc10_u0001-20221114_073700_36_241e_3B_Visual_2.npz'

In [9]:
from pathlib import Path
from tqdm.notebook import tqdm
import cv2
debug_path = Path("./debug")
debug_path.mkdir(exist_ok=True)

with open("/home/topkech/work/pytorch-superpoint/configs/cross_domain_train.yaml", "r") as f:
    config = yaml.safe_load(f)

cd = CrossDomain(task="train", **config["data"])
for i in tqdm(range(len(cd))):
    ex = cd[i]
    im = ex["image"][0][...,np.newaxis]
    im = np.dstack([1-ex["valid_mask"].squeeze()[..., np.newaxis], ex["labels_2D_gaussian"][0][...,np.newaxis], im])

    warped_im = ex["warped_img"][0][...,np.newaxis]
    warped_im = np.dstack([1-ex["warped_valid_mask"].squeeze()[..., np.newaxis], ex["warped_labels_gaussian"][0][...,np.newaxis], warped_im])
    imstack = np.clip(np.hstack([im, warped_im]) * 255, 0, 255).astype(np.uint8)
    imstack = cv2.cvtColor(imstack, cv2.COLOR_RGB2BGR)
    cv2.imwrite(str(debug_path/f"{ex['name']}.jpg"), imstack)


load labels from:  logs/trash/predictions/train


  0%|          | 0/606 [00:00<?, ?it/s]